# Data Preprocessing & Cleaning

This notebook covers **data cleaning, missing value handling, outlier smoothing, and preparation** 
for modeling.  

Steps:
1. Remove invalid or incomplete rows.
2. Handle missing values.
3. Smooth extreme/outlier values.
4. Sample / reduce numerosity for faster experiments.

In [1]:
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing & stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression

# Settings
pd.set_option("display.max_columns", None)
sns.set(style="whitegrid")

In [2]:
df = pd.read_csv("../data/interim/preprocessed.csv")
print("Shape:", df.shape)
df.head()

Shape: (1015341, 115)


,EXT1,EXT2,EXT3,EXT4,EXT5,EXT6,EXT7,EXT8,EXT9,EXT10,EST1,EST2,EST3,EST4,EST5,EST6,EST7,EST8,EST9,EST10,AGR1,AGR2,AGR3,AGR4,AGR5,AGR6,AGR7,AGR8,AGR9,AGR10,CSN1,CSN2,CSN3,CSN4,CSN5,CSN6,CSN7,CSN8,CSN9,CSN10,OPN1,OPN2,OPN3,OPN4,OPN5,OPN6,OPN7,OPN8,OPN9,OPN10,EXT1_E,EXT2_E,EXT3_E,EXT4_E,EXT5_E,EXT6_E,EXT7_E,EXT8_E,EXT9_E,EXT10_E,EST1_E,EST2_E,EST3_E,EST4_E,EST5_E,EST6_E,EST7_E,EST8_E,EST9_E,EST10_E,AGR1_E,AGR2_E,AGR3_E,AGR4_E,AGR5_E,AGR6_E,AGR7_E,AGR8_E,AGR9_E,AGR10_E,CSN1_E,CSN2_E,CSN3_E,CSN4_E,CSN5_E,CSN6_E,CSN7_E,CSN8_E,CSN9_E,CSN10_E,OPN1_E,OPN2_E,OPN3_E,OPN4_E,OPN5_E,OPN6_E,OPN7_E,OPN8_E,OPN9_E,OPN10_E,dateload,screenw,screenh,introelapse,testelapse,endelapse,IPC,country,lat_appx_lots_of_err,long_appx_lots_of_err,EXT,NEU,AGR,CON,OPN
0,4.0,1.0,5.0,2.0,5.0,1.0,5.0,2.0,4.0,1.0,1.0,4.0,4.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,5.0,2.0,4.0,2.0,3.0,2.0,4.0,3.0,4.0,3.0,4.0,3.0,2.0,2.0,4.0,4.0,2.0,4.0,4.0,5.0,1.0,4.0,1.0,4.0,1.0,5.0,3.0,4.0,5.0,9419.0,5491.0,3959.0,4821.0,5611.0,2756.0,2388.0,2113.0,5900.0,4110.0,6135.0,4150.0,5739.0,6364.0,3663.0,5070.0,5709.0,4285.0,2587.0,3997.0,4750.0,5475.0,11641.0,3115.0,3207.0,3260.0,10235.0,5897.0,1758.0,3081.0,6602.0,5457.0,1569.0,2129.0,3762.0,4420.0,9382.0,5286.0,4983.0,6339.0,3146.0,4067.0,2959.0,3411.0,2170.0,4920.0,4436.0,3116.0,2992.0,4354.0,2016-03-03 02:01:01,768.0,1024.0,9.0,234.0,6,1,GB,51.5448,0.1991,2329.90,2386.15,2622.50,2498.05,1780.20
1,3.0,5.0,3.0,4.0,3.0,3.0,2.0,5.0,1.0,5.0,2.0,3.0,4.0,1.0,3.0,1.0,2.0,1.0,3.0,1.0,1.0,4.0,1.0,5.0,1.0,5.0,3.0,4.0,5.0,3.0,3.0,2.0,5.0,3.0,3.0,1.0,3.0,3.0,5.0,3.0,1.0,2.0,4.0,2.0,3.0,1.0,4.0,2.0,5.0,3.0,7235.0,3598.0,3315.0,2564.0,2976.0,3050.0,4787.0,3228.0,3465.0,3309.0,9036.0,2406.0,3484.0,3359.0,3061.0,2539.0,4226.0,2962.0,1799.0,1607.0,2158.0,2090.0,2143.0,2807.0,3422.0,5324.0,4494.0,3627.0,1850.0,1747.0,5163.0,5240.0,7208.0,2783.0,4103.0,3431.0,3347.0,2399.0,3360.0,5595.0,2624.0,4985.0,1684.0,3026.0,4742.0,3336.0,2718.0,3374.0,3096.0,3019.0,2016-03-03 02:01:20,1360.0,768.0,12.0,179.0,11,1,MY,3.1698,101.706,1878.05,1725.00,1484.70,2133.00,1631.55
2,2.0,3.0,4.0,4.0,3.0,2.0,1.0,3.0,2.0,5.0,4.0,4.0,4.0,2.0,2.0,2.0,2.0,2.0,1.0,3.0,1.0,4.0,1.0,4.0,2.0,4.0,1.0,4.0,4.0,3.0,4.0,2.0,2.0,2.0,3.0,3.0,4.0,2.0,4.0,2.0,5.0,1.0,2.0,1.0,4.0,2.0,5.0,3.0,4.0,4.0,4657.0,3549.0,2543.0,3335.0,5847.0,2540.0,4922.0,3142.0,14621.0,2191.0,5128.0,3675.0,3442.0,4546.0,8275.0,2185.0,2164.0,1175.0,3813.0,1593.0,1089.0,2203.0,3386.0,1464.0,2562.0,1493.0,3067.0,13719.0,3892.0,4100.0,4286.0,4775.0,2713.0,2813.0,4237.0,6308.0,2690.0,1516.0,2379.0,2983.0,1930.0,1470.0,1644.0,1683.0,2229.0,8114.0,2043.0,6295.0,1585.0,2529.0,2016-03-03 02:01:56,1366.0,768.0,3.0,186.0,7,1,GB,54.9119,-1.3833,2368.80,1801.10,1850.15,1736.40,1477.65
3,2.0,2.0,2.0,3.0,4.0,2.0,2.0,4.0,1.0,4.0,3.0,3.0,3.0,2.0,3.0,2.0,2.0,2.0,4.0,3.0,2.0,4.0,3.0,4.0,2.0,4.0,2.0,4.0,3.0,4.0,2.0,4.0,4.0,4.0,1.0,2.0,2.0,3.0,1.0,4.0,4.0,2.0,5.0,2.0,3.0,1.0,4.0,4.0,3.0,3.0,3996.0,2896.0,5096.0,4240.0,5168.0,5456.0,4360.0,4496.0,5240.0,4000.0,3736.0,4616.0,3015.0,2711.0,3960.0,4064.0,4208.0,2936.0,7336.0,3896.0,6062.0,11952.0,1040.0,2264.0,3664.0,3049.0,4912.0,7545.0,4632.0,6896.0,2824.0,520.0,2368.0,3225.0,2848.0,6264.0,3760.0,10472.0,3192.0,7704.0,3456.0,6665.0,1977.0,3728.0,4128.0,3776.0,2984.0,4192.0,3480.0,3257.0,2016-03-03 02:02:02,1920.0,1200.0,186.0,219.0,7,1,GB,51.75,-1.25,2248.70,2025.25,2602.40,2160.20,1883.70
4,3.0,3.0,3.0,3.0,5.0,3.0,3.0,5.0,3.0,4.0,1.0,5.0,5.0,3.0,1.0,1.0,1.0,1.0,3.0,2.0,1.0,5.0,1.0,5.0,1.0,3.0,1.0,5.0,5.0,3.0,5.0,1.0,5.0,1.0,3.0,1.0,5.0,1.0,5.0,5.0,5.0,1.0,5.0,1.0,5.0,1.0,5.0,3.0,5.0,5.0,6004.0,3965.0,2721.0,3706.0,2968.0,2426.0,7339.0,3302.0,16819.0,3731.0,4740.0,2856.0,7461.0,2179.0,3324.0,2255.0,4308.0,4506.0,3127.0,3115.0,6771.0,2819.0,3682.0,2511.0,16204.0,1736.0,28983.0,1612.0,2437.0,4532.0,3843.0,7019.0,3102.0,3153.0,2869.0,6550.0,1811.0,3682.0,21500.0,20587.0,8458.0,3510.0,17042.0,7029.0,2327.0,5835.0,6846.0,5320.0,11401.0,8642.0,2016-03-03 02:02:57,1366.0,768.0,8.0,315.0,17,2,KE,1.0,38.0,2

In [3]:
# Filter: remove rows with extreme invalid responses (all zeros, all 5s, or missing > 20%)
trait_cols = ["EXT", "NEU", "AGR", "CON", "OPN"]

# Drop rows with NaN in any Big Five trait
df_clean = df.dropna(subset=trait_cols)

# Optional: Remove rows with 0 or 5 in all trait columns (straight-liners)
mask_straightline = ((df_clean[trait_cols] == 0).all(axis=1)) | ((df_clean[trait_cols] == 5).all(axis=1))
df_clean = df_clean[~mask_straightline]

print("✅ Shape after removing invalid/incomplete rows:", df_clean.shape)

✅ Shape after removing invalid/incomplete rows: (1011478, 115)


In [4]:
# Impute numeric features (items) using median
item_cols = [c for c in df_clean.columns if c.startswith(("EXT","EST","AGR","CSN","OPN")) and "_E" not in c]
imputer = SimpleImputer(strategy="median")
df_clean[item_cols] = imputer.fit_transform(df_clean[item_cols])

# Impute categorical/object columns using mode
cat_cols = df_clean.select_dtypes(include="object").columns
for col in cat_cols:
    df_clean[col].fillna(df_clean[col].mode()[0], inplace=True)

print("✅ Missing values handled")
print(df_clean[item_cols].isnull().sum().sum())  # Should be 0

C:\Users\ASUS\AppData\Local\Temp\ipykernel_33464\2806768816.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean[col].fillna(df_clean[col].mode()[0], inplace=True)


✅ Missing values handled
0


In [5]:
# Apply boundary smoothing for trait scores (cap/floor)
for col in trait_cols:
    df_clean[col] = df_clean[col].clip(lower=df_clean[col].quantile(0.01),
                                      upper=df_clean[col].quantile(0.99))

# Optional: bin traits into 3 categories for later classification
for col in trait_cols:
    df_clean[f"{col}_bin"] = pd.qcut(df_clean[col], q=3, labels=["Low","Medium","High"])

print("✅ Smoothed extreme values and added discretized bins")

✅ Smoothed extreme values and added discretized bins


In [6]:
df_clean.to_csv("../data/interim/cleaned.csv", index=False)
print("✅ Saved cleaned dataset to data/interim/cleaned.csv")

✅ Saved cleaned dataset to data/interim/cleaned.csv


In [7]:
# StandardScaler for ML models that require normalized inputs
scaler = StandardScaler()
df_clean[item_cols] = scaler.fit_transform(df_clean[item_cols])

print("✅ Standardized item responses")

✅ Standardized item responses


In [8]:
# Aggregate features: mean & std per trait group
trait_item_groups = {
    "EXT": [f"EXT{i}" for i in range(1,11)],
    "NEU": [f"EST{i}" for i in range(1,11)],
    "AGR": [f"AGR{i}" for i in range(1,11)],
    "CON": [f"CSN{i}" for i in range(1,11)],
    "OPN": [f"OPN{i}" for i in range(1,11)]
}

for trait, items in trait_item_groups.items():
    df_clean[f"{trait}_mean"] = df_clean[items].mean(axis=1)
    df_clean[f"{trait}_std"] = df_clean[items].std(axis=1)

print("✅ Added per-trait mean and std features")

✅ Added per-trait mean and std features


In [9]:
# For faster experimentation, sample a subset (e.g., 100k rows)
df_sample = df_clean.sample(n=100000, random_state=42)
print("✅ Sampled 100k rows for fast experiments")

✅ Sampled 100k rows for fast experiments


In [10]:
df_clean.to_csv("../data/processed/feature_engineered.csv", index=False)
print("✅ Saved processed dataset to data/processed/feature_engineered.csv")

✅ Saved processed dataset to data/processed/feature_engineered.csv
